In [27]:
inference_server_url = "INFERENCE_SERVER_URL"
redis_url = "REDIS_SERVER_URL"
index_name = "docs"
schema_name = "redis_schema.yaml"

In [28]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.redis import Redis
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [29]:
embeddings = HuggingFaceEmbeddings()
rds = Redis.from_existing_index(
    embeddings,
    redis_url=redis_url,
    index_name=index_name,
    schema=schema_name
)

In [50]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
You will be given a question you need to answer, and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Question: {question}

Context: {context} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [48]:
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain = RetrievalQA.from_chain_type(llm,
           retriever=rds.as_retriever(search_type="similarity", search_kwargs={"k": 4, "distance_threshold": 0.5}),
           chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
           return_source_documents=True)

In [57]:
def build_query(product):
    query = """
        Write a detailed product description for {product} that includes its name, company, technology, design features, and any additional information that would be relevant to a potential customer.
        Highlight the benefits of the {product} and explain how it can be used for various projects.
        Got 4 out 5 star review from clients. To request demo, contact seller,include specifications and use.
        The blog structure should have a title, headings with multiple Subheadings, conclusion and a call-to-action section as necessary.
        Use Modern Language Association (MLA) style with a formal tone. The length of the blog post should be a minimum of 1800 words
    """
    return query.format(product=product)

result=qa_chain({"query":build_query("OpenShift")})


# OpenShift Data Science Overview

OpenShift is an open-source container application platform that enables developers to build, deploy, and manage applications in a cloud or on-premises environment. It provides a comprehensive set of tools for developing, testing, and deploying applications across multiple environments. In this chapter, we will provide an overview of OpenShift data science features and capabilities.

## Supported Configurations

OpenShift supports various configurations for data science workloads, including:

* **On-Premises**: Developers can run OpenShift clusters on their own infrastructure, providing full control over the underlying resources and security policies. This configuration allows organizations to leverage existing hardware investments while maintaining flexibility and scalability.
* **Public Cloud**: OpenShift offers managed services on popular public clouds such as Amazon Web Services (AWS), Microsoft Azure, and Google Cloud Platform (GCP). These cloud 

In [59]:
print(result)

{'query': '\n        Write a detailed product description for OpenShift that includes its name, company, technology, design features, and any additional information that would be relevant to a potential customer.\n        Highlight the benefits of the OpenShift and explain how it can be used for various projects.\n        Got 4 out 5 star review from clients. To request demo, contact seller,include specifications and use.\n        The blog structure should have a title, headings with multiple Subheadings, conclusion and a call-to-action section as necessary.\n        Use Modern Language Association (MLA) style with a formal tone. The length of the blog post should be a minimum of 1800 words\n    ', 'result': '\n# OpenShift Data Science Overview\n\nOpenShift is an open-source container application platform that enables developers to build, deploy, and manage applications in a cloud or on-premises environment. It provides a comprehensive set of tools for developing, testing, and deployin